In [12]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import wandb

In [13]:
wandb.login()
api = wandb.Api()
artifact = api.artifact('rueedi-tobias-hochschule-luzern/Own CNN/cnn-model:v3', type='model')
model_path = artifact.download()

for file in os.listdir(model_path):
    if file.endswith(".h5") or file.endswith(".keras"):
        model_big = tf.keras.models.load_model(os.path.join(model_path, file), compile=False)
        break

artifact = api.artifact('rueedi-tobias-hochschule-luzern/Own CNN/cnn-model:v4', type='model')
model_path = artifact.download()

for file in os.listdir(model_path):
    if file.endswith(".h5") or file.endswith(".keras"):
        model_small = tf.keras.models.load_model(os.path.join(model_path, file), compile=False)
        break



wandb: Downloading large artifact cnn-model:v3, 50.21MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7 (76.4MB/s)
wandb:   1 of 1 files downloaded.  


In [14]:
# --- Parameters (tweak these to your dataset) ---
IMG_SIZE    = 640
GRID_SIZE   = 20
NUM_CLASSES = 1   #
NUM_BOXES   = 3    # number of anchors per cell
OUTPUT_SHAPE = (GRID_SIZE, GRID_SIZE, NUM_BOXES * (5 + NUM_CLASSES))

In [15]:
# --- YOLO loss with binary class ---
def yolo_loss(y_true, y_pred):
    # reshape to (..., S, S, B, 5 + C)
    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    obj_mask = true[..., 4:5]

    # localization losses
    xy_loss = tf.reduce_sum(obj_mask * tf.square(true[..., 0:2] - pred[..., 0:2]))
    wh_loss = tf.reduce_sum(obj_mask * tf.square(
        tf.sqrt(true[..., 2:4] + 1e-6) - tf.sqrt(pred[..., 2:4] + 1e-6)
    ))

    # objectness and class losses
    obj_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(true[..., 4:5], pred[..., 4:5])
    )
    class_bce = tf.keras.losses.binary_crossentropy(true[..., 5:], pred[..., 5:])
    class_bce = tf.expand_dims(class_bce, axis=-1)
    class_loss = tf.reduce_sum(obj_mask * class_bce)

    return xy_loss + wh_loss + obj_loss + class_loss

In [16]:
def area_difference_loss(y_true, y_pred):

    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))


    pred_area = pred[..., 2] * pred[..., 3] * pred[..., 4]
    true_area = true[..., 2] * true[..., 3] * true[..., 4]


    total_pred_area = tf.reduce_sum(pred_area)
    total_true_area = tf.reduce_sum(true_area)


    return tf.abs(total_pred_area - total_true_area)

In [17]:
def combined_yolo_area_loss(y_true, y_pred, area_weight=0.01):

    yolo = yolo_loss(y_true, y_pred)

    area = area_difference_loss(y_true, y_pred)

    return yolo + area_weight * area

In [18]:
def ensemble_yolo_outputs(pred1, pred2, weights=(0.4, 0.4, 0.2)):
    """
    Averages YOLO grid outputs from 3 models with specified weights.
    Each pred has shape: [GRID, GRID, NUM_BOXES, 5 + NUM_CLASSES]
    """
    # Ensure the shape is identical
    assert pred1.shape == pred2.shape 

    # Weighted average across predictions
    return (
        weights[0] * pred1 +
        weights[1] * pred2 
    )


In [19]:
def evaluate_ensemble_on_testset(model1, model2, test_ds, loss_fn, weights=(0.5, 0.5)):
    total_loss = 0.0
    num_batches = 0

    for batch in test_ds:
        images, labels = batch
        pred1 = model1.predict(images)
        pred2 = model2.predict(images)
        # Ensemble: shape [B, GRID, GRID, 3, 6]
        combined_pred = (
            weights[0] * pred1 +
            weights[1] * pred2 
        )

        # Evaluate batch loss
        loss = loss_fn(labels, combined_pred)
        total_loss += loss.numpy()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Ensemble {loss_fn.__name__} on test set: {avg_loss:.4f}")
    return avg_loss


In [20]:
def parse_tfrecord(example_proto):
    features = {
        "image_raw": tf.io.FixedLenFeature([], tf.string),
        "label_raw": tf.io.FixedLenFeature([], tf.string),
        "filename": tf.io.FixedLenFeature([], tf.string)
    }
    parsed = tf.io.parse_single_example(example_proto, features)
    image = tf.io.decode_jpeg(parsed["image_raw"], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.io.parse_tensor(parsed["label_raw"], out_type=tf.float32)
    label = tf.ensure_shape(label, [GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES])
    return image, label

def load_test_dataset_from_tfrecord(tfrecord_path, batch_size=4):
    ds = tf.data.TFRecordDataset(tfrecord_path)
    ds = ds.map(parse_tfrecord)
    return ds.batch(batch_size)


In [21]:
test_ds = load_test_dataset_from_tfrecord(r"C:\Users\samue\OneDrive\AIML\FS2025\DSPRO\M-AI-ZE-Maize-diseases-detection\data\test_dataset_20.tfrecord")

In [22]:
evaluate_ensemble_on_testset(
    model_big, model_small,
    test_ds=test_ds,
    loss_fn=combined_yolo_area_loss,  # your custom loss
    weights=(0.5, 0.5)
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 945ms/st ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/st ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/st ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 261

np.float32(70.25956)